In [1]:
# librairie globales
import io
from io import StringIO
from os import listdir
from os.path import isfile, join
from pathlib import Path
import pandas

# pour convert_pdf_to_txt
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import HTMLConverter,TextConverter,XMLConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage

# pour prepare_sentence
import nltk, os, subprocess, code, glob, re, traceback, sys, inspect
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize

# pour extract specific text
from io import BytesIO
import re

[nltk_data] Downloading package wordnet to /home/thomas/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/thomas/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/thomas/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/thomas/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [40]:
class extract_pdf:
    
    def __init__(self, path, langage="french"):
        self.path = path
        self.allfiles = [f for f in listdir(self.path) if isfile(join(self.path, f))]
        print(self.allfiles)
        self.langage = langage
        
        
    def convert_pdf_to_txt(self):

        pdf_folder = Path(self.path)
        # print(self.allfiles)
        list_txt = list()
        for files in self.allfiles:
            rsrcmgr = PDFResourceManager()
            retstr = BytesIO()
            codec = 'utf-8'
            word_margin = 0.1
            line_margin = 0.1
            char_margin = 0.5
            laparams = LAParams(word_margin=word_margin, line_margin=line_margin, char_margin=char_margin)
            device = HTMLConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
            print(files)
            fp = open(pdf_folder / files, 'rb')
            interpreter = PDFPageInterpreter(rsrcmgr, device)
            password = ""
            maxpages = 0
            caching = True
            pagenos = set()
            #text = files
            for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password, caching=caching,
                                          check_extractable=True):
                interpreter.process_page(page)
            
            text = retstr.getvalue()
            text =text.decode('utf-8')
            print("Longueur du doc : ")
            print(len(text))
            print("Début du doc : ")
            print(text[1:100])
            list_txt.append(text)
            print(len(list_txt))
            fp.close()
            device.close()
            retstr.close()

        return list_txt
      
        
    
    def prepare_sentence(self, list_txt):
        
        stop_words = set(stopwords.words(self.langage))
        clean_text_list = list()
        #Retrait de la ponctuation
        #text = re.sub('[^a-zA-Z]', ' ', sentence)
        for sentence in list_txt:
            print("Longueur du doc de base : ")
            print(len(sentence))
            print("Début du doc de base : ")
            print(sentence[1:100])
            #Conversion en minuscule
            text = sentence.lower()

            #Retrait des tags
            text=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",text)

            #Retrait des numéros et caractères spéciaux
            text=re.sub("(\\d|\\W)+"," ",text)

            #Conversion en liste
            text = text.split()


            text = [word for word in text if not word in  
                    stop_words] 
            text = " ".join(text)
            print("Longueur du doc : ")
            print(len(text))
            print("Début du doc : ")
            print(text[1:100])
            clean_text_list.append(text)
        return clean_text_list
    
    def export_text(self, list_txt):
        
        for sentence in list_txt:
            text_file = open("./sample.txt", "w")
            n = text_file.write(sentence)
            text_file.close()
    
    def extract_specific_text(self, list_txt, type_of_text,news ):
        if news == '20min':
            if type_of_text == 'title':
                regex = r'HelveticaNeueLTStd-Md\'; font-size:41px">(.*?)\n<br></span></div>'
            if type_of_text == 'paragraph':
                regex = r'DINOT-Regular\'; font-size:11px">(.*?)</span><span style="font-family'
        if news == 'cnews':
            if type_of_text == 'title':
                regex = r'(Canalp-ReguItal\'; font-size:50px">(.*?)\n<br></span></div>) | (Canalp-Regu\'; font-size:52px">(.*?)\n<br></span></div>)'
            if type_of_text == 'paragraph':
                regex = r'FrutigerLTCom-Condensed\'; font-size:9px">(.*?)</span><span style="font-family'
            
        list_specific_t = list()
        for sentence in list_txt:
            elements_to_grep = re.findall( regex, sentence,re.DOTALL)
            cleanr = re.compile('<.*?>|\n')
            for text_to_clean in elements_to_grep:
                clean_text = re.sub(cleanr, '', text_to_clean)
                list_specific_t.append(clean_text)
            #list_specific_t.append(elements_to_grep)
            return list_specific_t

In [41]:
extract20min = extract_pdf('/home/thomas/Documents/mediaData/echantillons_pdf/')
text_html = extract20min.convert_pdf_to_txt()
title20min = extract20min.extract_specific_text(text_html,'title',news = 'cnews')
paragraph20min = extract20min.extract_specific_text(text_html,'paragraph',news = 'cnews')


['CNEWS-20190927-0.pdf']
CNEWS-20190927-0.pdf
Longueur du doc : 
1455444
Début du doc : 
html><head>
<meta http-equiv="Content-Type" content="text/html; charset=utf-8">
</head><body>
<span
1


In [42]:
paragraph20min = extract20min.extract_specific_text(text_html,'paragraph',news = 'cnews')
paragraph20min

['certaine expression sur jouit d’une rue».  Une triangulaire,  qui  soutient Lorsqu’un commerçant ',
 'ou uneentreprise notoriété, on peut dire qu’il a «pi-gnon leapparue  au  Moyen Age,  dans domaine de l’architecture. Le mot«pignon» désigne en effet la par-tie haute d’un mur extérieur, par-fois lacharpente d’une maison. Les fa-l’époque  souventçades  étant  à construites lesdeux termes ont ﬁni par deveniremployaitsynonymes.  Ainsi, l’expression surrue» pour dire que sa façade prin-cipale  ou  que  sa  vitrine  donnaitsur la rue. Un luxe qui n’était alorspignon «avoir façon, cette on de L’IMAGEL’HISTOIREQUAND LE PRÊTRE MONTRE SES MUSCLESLe dieu du ﬁtness ? ',
 'Un pasteur sué-dois, Oskar Arngarden, est récemmentdevenu un phénomène sur les réseauxsociaux. Et pour cause, l’homme d’Eglisevidéos y  partage  des  photos  et  des de ses séances de musculation, qui se mêlent à des clichés de lui en train debaptiser, marier ou encore célébrer desofﬁces.  Un  cocktail  détonant, loin  deAPIS/